In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd

## Working code

In [5]:
driver = webdriver.Chrome()

# URL of MagicBricks PG hostels in Hyderabad
url = "https://www.magicbricks.com/property-for-rent/residential-paying-guest?city=2060"

driver.get(url)
time.sleep(5)  # Wait for the initial load

pg_hostels = []
scroll_pause_time = 5  # Pause to wait for loading new data
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load new content
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
    # Get page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Extract data
    listings = soup.find_all('div', class_='m-srp-card clearfix newLaunched')  # Update class name based on the actual HTML structure
    for listing in listings:
        try:
            title = listing.find('span', class_='m-srp-card__title__name').text.strip()
        except:
            title = None
    
        try:
            location = listing.find('div', class_='m-srp-card__link m-srp-card__link--nearby').text.strip()
        except:
            location = None
    
        try:
            price = listing.find('div', class_='m-srp-card__price').text.strip()
        except:
            price = None

        try:
            room_info = listing.find('div', class_='m-srp-card__roomInfo__room-detail clearfix').text.strip()
        except:
            price = None

        try:
            type = listing.find('div', class_='m-srp-card__info flex__item').text.strip()
        except:
            price = None

        try:
            places = listing.find('div', class_='m-srp-card__tuple__flex').text.strip()
        except:
            places = None
    
        pg_hostels.append({
            'Title': title,
            'Location': location,
            'Price': price,
            'Room Info':room_info,
            'Type':type,
            'Places':places
        })

# Convert to DataFrame and save to CSV
df = pd.DataFrame(pg_hostels)

# Close the WebDriver
driver.quit()

print("Scraping complete.")
df

Scraping complete.


,Title,Location,Price,Room Info,Type,Places
0,N GRAND MEN'S PG/Paying Guest,in Kondapur,"₹10,000 Onwards","Single Room ₹22,000 Twin Sharing ₹12,000 Tri...","Boys₹10,000 OnwardsFood IncludedBeds Available...",0.2 Km from Chirec International School 1.4 Km...
1,Sugathi Hometel PG/Paying Guest For Women,in Nanakaramguda,"₹7,000 Onwards","Single Room ₹16,000 Twin Sharing ₹9,000 Trip...","Girls₹7,000 OnwardsFood IncludedBeds Available...",0.9 Km from Isb (Indian School Of Business) 1....
2,MS Comforts Women's PG/Paying Guest,in Gachibowli,"₹8,000 Onwards","Single Room ₹18,000 Twin Sharing ₹10,000 Tri...","Girls₹8,000 OnwardsFood IncludedBeds Available...",0.3 Km from Asha Kiran 0.6 Km from Dlf Cyberci...
3,SKY LINE CO LIVING PG/Paying Guest,in Gachibowli,"₹18,000 Onwards","Single Room With AC ₹30,000 Twin Sharing With...","Coed₹18,000 OnwardsFood IncludedBeds Available...",0.9 Km from Isb (Indian School Of Business) 1....
4,Manjula Luxury PG/Paying Guest For MEN'S,in Gowlidoddy,"₹8,000 Onwards","Single Room ₹18,000 Twin Sharing ₹10,000 Tri...","Boys₹8,000 OnwardsFood IncludedBeds Available ...",1.6 Km from Isb (Indian School Of Business) 1....
...,...,...,...,...,...,...
975,My Spaces Luxury Men's PG/Paying Guest,in Kondapur,"₹9,500 Onwards","Single Room ₹22,000 Twin Sharing ₹12,000 Tri...","Boys₹9,500 OnwardsFood IncludedBeds Available ...",0.9 Km from World One School 1.9 Km from Kohin...
976,Estay Executive Men's PG/Paying Guest,in Hitech City,"₹7,500 Onwards","Single Room With AC ₹22,000 Twin Sharing With...","Boys₹7,500 OnwardsFood IncludedBeds Available ...",0.6 Km from Sri Venkateshwara Group Of College...
977,R3 ATMOS LIVE THE SWAG PG/Paying Guest,in Hitex Road,"₹11,000 Onwards","Single Room With AC ₹28,000 Twin Sharing With...","Coed₹11,000 OnwardsFood IncludedBeds Available...",1.1 Km from Sarada High School 0.8 Km from Mod...
978,SVS Co living Stays PG/Paying Guest,in Kondapur,"₹9,000 Onwards","Single Room With AC ₹18,000 Twin Sharing With...","Coed₹9,000 OnwardsFood IncludedBeds Available ...",0.7 Km from Sarada High School 1.4 Km from Koh...


In [7]:
df.to_csv("pghostels.csv")